Qi Mao

Haowen Luo

In [255]:
from keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate, Dropout, Activation
from keras.models import Model
from keras.optimizers import Adam
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.regularizers import l2
from math import sqrt
import numpy as np
import pandas as pd
import matplotlib as plt

In [69]:
ratings= pd.read_csv('/Users/kindred/csci5123/a3-code/ml-1m/ratings.dat',sep='::',header=None)

/Users/kindred/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [70]:
ratings=ratings.rename(columns={0: "user", 1: "item",2: "rating",3: "timestamp"})
ratings=ratings[['user','item','rating']]
ratings.head()

,user,item,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


<h2>FunkSVD<b2>

In [99]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.batch import predict
from lenskit.algorithms.funksvd import FunkSVD
from lenskit.metrics.predict import rmse
from lenskit.algorithms import Recommender

In [100]:
def eval1(aname, algo, train, test):
    algo.fit(train)
    preds = predict(algo, test)
    preds_train=predict(algo,train)
    return [aname,rmse(preds['prediction'],preds['rating']),rmse(preds_train['prediction'],preds_train['rating'])]

In [101]:
all_recs1 = []
test_data1 = []
algo_fsvd=FunkSVD(50)
for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
    test_data1.append(test)
    all_recs1.append(eval1('FunkSVD', algo_fsvd, train, test))

In [102]:
df = pd.DataFrame(np.array(all_recs1))
df=df.rename(columns={0: "algorithm", 1: "RSME",2:"RSME_for_train"})
df['RSME'] = df['RSME'].astype(float)
df['RSME_for_train'] = df['RSME_for_train'].astype(float)

In [105]:
df.groupby('algorithm').RSME.mean()

algorithm
FunkSVD    0.854077
Name: RSME, dtype: float64

<h2>Keras 

In [369]:
def Rmse(y_actual,y_predicted):
    return (sqrt(mean_squared_error(y_actual, y_predicted)))

In [71]:
ratings['user_id']=ratings['user']
ratings['item_id']=ratings['item']

In [365]:
ratings.head()

,user,item,rating,user_id,item_id
0,1,1193,5,1,1193
1,1,661,3,1,661
2,1,914,3,1,914
3,1,3408,4,1,3408
4,1,2355,5,1,2355


In [308]:
dataset=ratings.copy()
#https://nipunbatra.github.io/blog/2017/recommend-keras.html In[5]
dataset['user_id'] = dataset['user_id'].astype('category').cat.codes.values
dataset['item_id'] = dataset['item_id'].astype('category').cat.codes.values
#https://github.com/ririw/ririw.github.io/blob/master/assets/Recommending%20movies.ipynb In[6]
y = np.zeros((dataset.shape[0],5))
y[np.arange(dataset.shape[0]),dataset.rating -1]=1

In [364]:
dataset.head()

,user,item,rating,user_id,item_id
0,1,1193,5,0,1104
1,1,661,3,0,639
2,1,914,3,0,853
3,1,3408,4,0,3177
4,1,2355,5,0,2162


In [310]:
train, test, train_y, test_y= train_test_split(dataset,y, test_size=0.2)

In [311]:
n_user, n_item = len(dataset['user'].unique()), len(dataset['item'].unique())
print(str(n_user)+"\n"+str(n_item))

6040
3706


In [312]:
#First of all, I thought of using five output nodes to output the results, which represent 1-5 ratings. 
#But with this method, the RMSE will be very large, even exceeding 1, but the MAE will be very small.
def Recommender(user,item):
    n_latent_factors_user = 50
    n_latent_factors_item = 50

    item_input = Input(shape=[1],name='Item')
    item_embedding = Embedding(item+1, n_latent_factors_item, name='Item-Embedding')(item_input)
    item_vec = Flatten(name='FlattenItem')(item_embedding)
    item_vec = Dropout(0.2)(item_vec)


    user_input = Input(shape=[1],name='User')
    user_embedding = Embedding(user+1, n_latent_factors_user, name='User-Embedding')(user_input)
    user_vec = Flatten(name='FlattenUsers')(user_embedding)
    user_vec = Dropout(0.2)(user_vec)

    #http://blog.richardweiss.org/2016/09/25/movie-embeddings.html
    concat = Concatenate()([item_vec,user_vec])
    lm = Dropout(0.2)(concat)
    lm = Dense(200,name='FullyConnected', activation='relu')(lm)
    lm = Dropout(0.2,name='Dropout1')(lm)
    lm = Dense(100,name='FullyConnected-1', activation='relu')(lm)
    lm = Dropout(0.3,name='Dropout2')(lm)
    lm = Dense(50,name='FullyConnected-2',activation='relu')(lm)
    lm = Dropout(0.4,name='Dropout3')(lm)
    lm = Dense(5,name='FullyConnected-4',activation='softmax')(lm)
    

    model = Model([item_input,user_input], lm)
    model.compile(optimizer='adam',loss= 'categorical_crossentropy')
    return model

In [313]:
model = Recommender(n_user, n_item)
model.summary()

Model: "model_46"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Item (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
User (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
Item-Embedding (Embedding)      (None, 1, 50)        185350      Item[0][0]                       
__________________________________________________________________________________________________
User-Embedding (Embedding)      (None, 1, 50)        302050      User[0][0]                       
___________________________________________________________________________________________

In [315]:
history = model.fit([train['item_id'],train['user_id'] ], train_y, epochs=5, verbose=1)

Epoch 1/5
800167/800167 [==============================] - 124s 155us/step - loss: 1.2705
Epoch 2/5
800167/800167 [==============================] - 126s 157us/step - loss: 1.2298
Epoch 3/5
800167/800167 [==============================] - 129s 161us/step - loss: 1.2205
Epoch 4/5
800167/800167 [==============================] - 139s 174us/step - loss: 1.2142
Epoch 5/5
800167/800167 [==============================] - 132s 165us/step - loss: 1.2101


In [370]:
print("RMSE is ",Rmse(np.argmax(test_y,1)+1, np.argmax(model.predict([test['item_id'], test['user_id']]),1)+1))
print("MSE is ",mean_squared_error(np.argmax(test_y,1)+1, np.argmax(model.predict([test['item_id'], test['user_id']]),1)+1))
print("MAE is ",mean_absolute_error(np.argmax(test_y,1)+1, np.argmax(model.predict([test['item_id'], test['user_id']]),1)+1))

RMSE is  1.0167388606907497
MSE is  1.0337579108387238
MAE is  0.677412743323902


RMSE is much bigger than FunkSVD

In [342]:
#This time, I chose to remove too many dropouts, and finally use the relu activation function according to 
#https://nipunbatra.github.io/blog/2017/recommend-keras.html
def Recommender_opt(user,item):
    n_latent_factors_user = 50
    n_latent_factors_item = 50

    item_input = Input(shape=[1],name='Item')
    item_embedding = Embedding(item+1, n_latent_factors_item, name='Item-Embedding')(item_input)
    item_vec = Flatten(name='FlattenItem')(item_embedding)
    item_vec = Dropout(0.2)(item_vec)


    user_input = Input(shape=[1],name='User')
    user_embedding = Embedding(user+1, n_latent_factors_user, name='User-Embedding')(user_input)
    user_vec = Flatten(name='FlattenUsers')(user_embedding)
    user_vec = Dropout(0.2)(user_vec)



    concat = Concatenate()([item_vec,user_vec])
    lm = Dropout(0.2)(concat)
    lm = Dense(200,name='FullyConnected')(lm)
    lm = Dense(100,name='FullyConnected-1')(lm)
    lm = Dense(50,name='FullyConnected-2')(lm)
    lm = Dense(20,name='FullyConnected-3',activation='relu')(lm)
    lm = Dense(1,name='FullyConnected-4')(lm)
    
    result = Activation('relu')(lm)
    opt = Adam(lr=0.001)
    
    model = Model([user_input, item_input], result)
    model.compile(optimizer=opt,loss= 'mean_squared_error')
    return model

In [343]:
model_opt = Recommender_opt(n_user, n_item)

In [344]:
model_opt.summary()

Model: "model_51"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Item (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
User (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
Item-Embedding (Embedding)      (None, 1, 50)        185350      Item[0][0]                       
__________________________________________________________________________________________________
User-Embedding (Embedding)      (None, 1, 50)        302050      User[0][0]                       
___________________________________________________________________________________________

In [345]:
history = model_opt.fit([train['user_id'],train['item_id'] ], train['rating'], epochs=10, verbose=1)

/Users/kindred/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
800167/800167 [==============================] - 130s 162us/step - loss: 0.9058
Epoch 2/10
800167/800167 [==============================] - 171s 214us/step - loss: 0.8363
Epoch 3/10
800167/800167 [==============================] - 173s 216us/step - loss: 0.8179
Epoch 4/10
800167/800167 [==============================] - 165s 206us/step - loss: 0.8068
Epoch 5/10
800167/800167 [==============================] - 164s 204us/step - loss: 0.7990
Epoch 6/10
800167/800167 [==============================] - 160s 199us/step - loss: 0.7940
Epoch 7/10
800167/800167 [==============================] - 152s 190us/step - loss: 0.7897
Epoch 8/10
800167/800167 [==============================] - 154s 192us/step - loss: 0.7860
Epoch 9/10
800167/800167 [==============================] - 139s 173us/step - loss: 0.7821
Epoch 10/10
800167/800167 [==============================] - 162s 203us/step - loss: 0.7793


In [371]:
print("RMSE is ",Rmse(test['rating'], model_opt.predict([test['user_id'], test['item_id']])))

RMSE is  0.8865349192389449


In [372]:
print("MSE is ",mean_squared_error(test['rating'], model_opt.predict([test.user_id, test.item_id])))

MSE is  0.7859441630300026


In [373]:
print("MAE is ",mean_absolute_error(test['rating'], model_opt.predict([test.user_id, test.item_id])))

MAE is  0.6974150409064045


In [358]:
#To minimize RMSE, I chose to fit the model as much as possible, or even overfit. so I add two layers
def Recommender_opt2(user,item):
    n_latent_factors_user = 50
    n_latent_factors_item = 50

    item_input = Input(shape=[1],name='Item')
    item_embedding = Embedding(item+1, n_latent_factors_item, name='Item-Embedding')(item_input)
    item_vec = Flatten(name='FlattenItem')(item_embedding)



    user_input = Input(shape=[1],name='User')
    user_embedding = Embedding(user+1, n_latent_factors_user, name='User-Embedding')(user_input)
    user_vec = Flatten(name='FlattenUsers')(user_embedding)




    concat = Concatenate()([item_vec,user_vec])
    lm = Dropout(0.2)(concat)
    lm = Dense(400,name='FullyConnected',activation='relu')(lm)
    lm = Dense(200,name='FullyConnected-1',activation='relu')(lm)
    lm = Dense(100,name='FullyConnected-2',activation='relu')(lm)
    lm = Dense(50,name='FullyConnected-3',activation='relu')(lm)
    lm = Dense(20,name='FullyConnected-4',activation='relu')(lm)
    lm = Dense(10,name='FullyConnected-5',activation='relu')(lm)
    lm = Dense(1,name='FullyConnected-6')(lm)
    
    result = Activation('relu')(lm)
    opt = Adam(lr=0.001)
    
    model = Model([user_input, item_input], result)
    model.compile(optimizer=opt,loss= 'mean_squared_error')
    return model

In [359]:
model_opt2 = Recommender_opt2(n_user, n_item)
model_opt2.summary()

Model: "model_55"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Item (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
User (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
Item-Embedding (Embedding)      (None, 1, 50)        185350      Item[0][0]                       
__________________________________________________________________________________________________
User-Embedding (Embedding)      (None, 1, 50)        302050      User[0][0]                       
___________________________________________________________________________________________

In [360]:
history = model_opt2.fit([train['user_id'],train['item_id'] ], train['rating'], epochs=10, verbose=1)

/Users/kindred/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
800167/800167 [==============================] - 221s 276us/step - loss: 0.8857
Epoch 2/10
800167/800167 [==============================] - 217s 271us/step - loss: 0.7991
Epoch 3/10
800167/800167 [==============================] - 227s 284us/step - loss: 0.7722
Epoch 4/10
800167/800167 [==============================] - 211s 264us/step - loss: 0.7576
Epoch 5/10
800167/800167 [==============================] - 208s 260us/step - loss: 0.7473
Epoch 6/10
800167/800167 [==============================] - 212s 265us/step - loss: 0.7378
Epoch 7/10
800167/800167 [==============================] - 212s 265us/step - loss: 0.7273
Epoch 8/10
800167/800167 [==============================] - 212s 265us/step - loss: 0.7170
Epoch 9/10
800167/800167 [==============================] - 203s 254us/step - loss: 0.7070
Epoch 10/10
800167/800167 [==============================] - 211s 263us/step - loss: 0.6981


In [374]:
print("RMSE is ", Rmse(test['rating'], model_opt2.predict([test['user_id'], test['item_id']])))

RMSE is  0.8721426175498154


In [375]:
print("MSE is ", mean_squared_error(test['rating'], model_opt2.predict([test.user_id, test.item_id])))

MSE is  0.7606327453466435


In [376]:
print("MAE is ", mean_absolute_error(test['rating'], model_opt2.predict([test.user_id, test.item_id])))

MAE is  0.6790471602466616


<h2>reference 

https://nipunbatra.github.io/blog/2017/recommend-keras.html

https://medium.com/@jdwittenauer/deep-learning-with-keras-recommender-systems-e7b99cb29929

http://www.clungu.com/MovieLens/

http://blog.richardweiss.org/2016/09/25/movie-embeddings.html

https://github.com/ririw/ririw.github.io/blob/master/assets/Recommending%20movies.ipynb